In [1]:
%load_ext autoreload
%autoreload 2

In [14]:
from datapipeline import Datapipeline
import pandas as pd
import numpy as np
import pickle

In [3]:
import os
os.chdir('d:/Users/mokky/Documents/GitHub/nus-iss/PRS-PM-ISY5002-GROUP5/SystemCode')
print(os.getcwd())

d:\Users\mokky\Documents\GitHub\nus-iss\PRS-PM-ISY5002-GROUP5\SystemCode


## Get a sample from existing dataset

In [4]:
file_path ='./Data Exploration/data/ParkwaySampleDataForProject_09_withTOSP3.xlsx'
df = pd.read_excel(file_path)
print(df.shape)

(381211, 83)


In [5]:
import random
idx = random.choices(df.index)
print(idx)

[227831]


In [6]:
df_test = df.iloc[idx, :]
print(df_test.T)

                    227831
INSTITUTION            MEH
CASE_NUMBER     2019100838
TOTAL_PAID_AMT     49144.8
PAYER_CODE_1              
PAYER_NAME_1     Self-Paid
...                    ...
ICD_CODE3              NaN
ICDCODE_STRING        G932
PROF_FEE           21436.2
TOTAL_FEES         49144.8
WRITE_OFF                0

[83 rows x 1 columns]


In [7]:
print(df_test.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1 entries, 227831 to 227831
Data columns (total 83 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   INSTITUTION          1 non-null      object        
 1   CASE_NUMBER          1 non-null      int64         
 2   TOTAL_PAID_AMT       1 non-null      float64       
 3   PAYER_CODE_1         1 non-null      object        
 4   PAYER_NAME_1         1 non-null      object        
 5   PAYER_1_PAID_AMT     1 non-null      float64       
 6   PAYER_CODE_2         0 non-null      object        
 7   PAYER_NAME_2         0 non-null      object        
 8   PAYER_2_PAID_AMT     1 non-null      float64       
 9   PAYER_CODE_3         0 non-null      object        
 10  PAYER_NAME_3         0 non-null      object        
 11  PAYER_3_PAID_AMT     1 non-null      float64       
 12  PAYER_CODE_4         0 non-null      object        
 13  PAYER_NAME_4         0 non-nu

## Inference Code

### Initialisation

In [8]:
scaler_pkl_file_paths = {
    'GHL': f'./data/uc1/GHL_data_scaler.pkl',
    'MEH': f'./data/uc1/MEH_data_scaler.pkl',
    'PEH': f'./data/uc1/PEH_data_scaler.pkl',
    'PNH': f'./data/uc1/PNH_data_scaler.pkl',
}
ohe_pkl_file_paths = {
    'GHL': f'./data/uc1/GHL_data_ohe.pkl',
    'MEH': f'./data/uc1/MEH_data_ohe.pkl',
    'PEH': f'./data/uc1/PEH_data_ohe.pkl',
    'PNH': f'./data/uc1/PNH_data_ohe.pkl',
}
feature_importance_file_paths = {
    'GHL':f'./data/uc1/GHL_forest_feat_impt_uc1.npy',
    'MEH':f'./data/uc1/MEH_forest_feat_impt_uc1.npy',
    'PEH':f'./data/uc1/PEH_forest_feat_impt_uc1.npy',
    'PNH':f'./data/uc1/PNH_forest_feat_impt_uc1.npy',
}
model_file_paths = {
    'GHL':f'./models/uc1/GHL_uc1_1.0_smote_mlp_(256, 256)_model.pkl',
    'MEH':f'./models/uc1/MEH_uc1_1.0_smote_mlp_(256, 256)_model.pkl',
    'PEH':f'./models/uc1/PEH_uc1_1.0_smote_mlp_(256, 256)_model.pkl',
    'PNH':f'./models/uc1/PNH_uc1_1.0_smote_mlp_(256, 256)_model.pkl',
}

In [9]:
dpl = Datapipeline()

### Inference

In [20]:
dict_df_test_proc = dpl.transform_raw_test_data(df_test, split_hosp=True)
for hosp in dict_df_test_proc:
    df_test_new = dpl.transform_test_data(dict_df_test_proc[hosp].drop('WRITE_OFF', axis=1),
                                          scaler_pkl_file_paths[hosp],
                                          ohe_pkl_file_paths[hosp],
                                          feature_importance_file_paths[hosp])
    model = pickle.load(open(model_file_paths[hosp], 'rb'))
    y_pred = model.predict(df_test_new)
    print(f'Prediction ({hosp}):', y_pred[0]) #result
    print(f'Actual ({hosp}):', int(dict_df_test_proc[hosp]['WRITE_OFF'].values[0]))

Prediction (MEH): 0
Actual (MEH): 0
